In [0]:
%sql
USE CATALOG hive_metastore;

In [0]:
-- COMMAND ----------
-- Set Legacy Policy to handle any residual date string issues
SET spark.sql.legacy.timeParserPolicy = LEGACY;

-- Transformation and Loading to Target Table
INSERT INTO work.employee

WITH cleaned AS (
    SELECT
        -- Standard Cleaning
        LPAD(TRIM(Emp_ID), 8, '0')                        AS Emp_ID,
        REGEXP_REPLACE(TRIM(F_Name), '\\s+', '_')         AS F_Name,
        REGEXP_REPLACE(TRIM(L_Name), '\\s+', '_')         AS L_Name,
        LOWER(TRIM(Email))                                AS Email,
        
        -- Dates should now be correct from Rawz, but we Cast safely just in case
        CAST(Hire_Date AS DATE)                           AS Hire_Date,
        CAST(Birth_Date AS DATE)                          AS Birth_Date,
        
        REGEXP_REPLACE(Ph_Num, '[^0-9]', '')              AS Ph_Num,
        TRIM(Gender)                                      AS Gender,
        REGEXP_REPLACE(TRIM(Primary_Skill), '\\s+', '_')  AS Primary_Skill,
        TRIM(Designation)                                 AS Designation,
        TRIM(Training_Status)                             AS Training_Status,
        REGEXP_REPLACE(TRIM(Dept_Name), '\\s+', '_')      AS Dept_Name,
        LPAD(TRIM(Dept_ID), 8, '0')                       AS Dept_ID,
        
        -- Handle Managers
        CASE 
            WHEN Mang_ID IS NOT NULL AND UPPER(TRIM(Mang_ID)) NOT IN ('NULL', '')
            THEN LPAD(TRIM(Mang_ID), 8, '0') 
            ELSE NULL 
        END                                               AS Mang_ID,
        
        LPAD(TRIM(Training_ID), 8, '0')                   AS Training_ID,
        LPAD(TRIM(Proj_ID), 8, '0')                       AS Proj_ID,
        LPAD(TRIM(Payroll_ID), 8, '0')                    AS Payroll_ID,
        
        ingestTimestamp,
        loadKey
    FROM rawz.employee
),
filtered AS (
    SELECT *,
        CASE
            -- Filter Logic
            WHEN Email NOT RLIKE '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$' THEN 'Invalid Email'
            WHEN LENGTH(Ph_Num) < 7                                                 THEN 'Invalid Phone'
            WHEN Birth_Date IS NOT NULL AND Hire_Date IS NOT NULL 
                 AND Birth_Date >= Hire_Date                                        THEN 'Invalid Date Range'
            ELSE NULL
        END AS filter_reason,
        
        -- Checksum for Deduplication
        SHA2(
            CONCAT_WS('||', 
                Emp_ID, F_Name, L_Name, Email, Hire_Date, Birth_Date, 
                Ph_Num, Gender, Primary_Skill, Designation, Training_Status,
                Dept_Name, Dept_ID, COALESCE(Mang_ID, ''), 
                Training_ID, Proj_ID, Payroll_ID
            ), 256
        ) AS checksum
    FROM cleaned
),
ranked AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY checksum ORDER BY ingestTimestamp DESC) AS rn
    FROM filtered
)
SELECT 
    uuid() AS Emp_Key,
    Emp_ID,
    F_Name,
    L_Name,
    Email,
    Hire_Date,
    Birth_Date,
    Ph_Num,
    Gender,
    Primary_Skill,
    Designation,
    Training_Status,
    Dept_Name,
    Dept_ID,
    Mang_ID,
    Training_ID,
    Proj_ID,
    Payroll_ID,
    ingestTimestamp,
    loadKey,
    checksum
FROM ranked
WHERE filter_reason IS NULL 
  AND rn = 1;

-- COMMAND ----------
-- Verification: Check valid rows count
SELECT count(*) as Valid_Rows FROM work.employee;

In [0]:
select * from work.Employee;

In [0]:
select count(*) from work.employee;